In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import json
from threading import Thread,Lock,Event

In [2]:
headers = {
    'authority': 'www.bigbasket.com',
    'method': 'GET',
    'path': '/listing-svc/v2/products?type=pc&slug=foodgrains-oil-masala&page=1',
    'scheme': 'https',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Cookie': 'x-entry-context-id=100; x-entry-context=bb-b2c; _bb_locSrc=default; x-channel=web; _bb_loid=j:null; _bb_bhid=; _bb_nhid=1723; _bb_vid=MTI4MjA0NDYxMzQ=; _bb_dsevid=; _bb_dsid=; _bb_cid=1; _bb_aid=MzA4NTgxODk5Nw==; csrftoken=HhsvZkeZFNA7XdmHV5IIUT49QLIvvrXbMF4gCJoPWxShyovvBqwpje2fFCm84nEj; _bb_home_cache=4cca0b57.1.visitor; bb2_enabled=true; ufi=1; bigbasket.com=2f61e0d4-9d4f-4530-ba94-334049c46704; _gcl_au=1.1.2033074425.1718993968; _ga=GA1.1.1843003858.1718993968; jarvis-id=5d0d66fb-63a4-4685-a900-2dfc7b925de7; _ga_FRRYG5VKHX=GS1.1.1719002693.3.1.1719002693.60.0.0; csurftoken=egUF8w.MTI4MjA0NDYxMzQ=.1720255452916.XQjr+mZO4SgN4wjdV3jFFPEr7mvcgjkbbVDcgrCBe/w=',
    'Sec-Ch-Ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Microsoft Edge";v="126"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0',
    'Content-Type': 'application/json',
    'Newrelic': 'eyJ2IjpbMCwxXSwiZCI6eyJ0eSI6IkJyb3dzZXIiLCJhYyI6IjgzNzMwNCIsImFwIjoiMTgzNDk4NzAwMiIsImlkIjoiYmM1NzlmNzRiNjA1ZjkzYyIsInRyIjoiZWIzNTliYTAzZjg3NGFmNTA4MGI3YWY5NjRkN2QzNjIiLCJ0aSI6MTcyMDIyOTg3MTQyNH19',
    'Priority': 'u=1, i'
}

In [34]:
product_pages = []
lock = Lock()
flag=True
def scrap_page(page,lock):
    global flag
    url = f"https://www.bigbasket.com/listing-svc/v2/products?type=pc&slug=foodgrains-oil-masala&page={page}"
    try:
        response = requests.get(url, headers=headers)
        data = response.json()
        with lock:
            try:
                product_pages.append(data["tabs"][0]["product_info"].get("products",[]))
            except Exception as e:
                print(f"Error processing page {page}: {e}")
    except Exception as e:
        print(f"Request failed for page {page}: {e}")
        flag=False

if __name__ == "__main__":
    all_threads = []
    number_threads = 2
    page = 1
    while True:
        try:
            for _ in range(number_threads):
                t = Thread(target=scrap_page, args=(page,lock))
                if not flag :
                    raise Exception("Page Limit Reached")
                all_threads.append(t)
                t.start()
                # print(page)
                page += 1
                
            for t in all_threads:
                t.join()
            
            all_threads = []
        except Exception as e:
            print(f"Total pages scraped: {len(product_pages)}")
            # print(product_info)
            break

Request failed for page 276: Expecting value: line 1 column 1 (char 0)
Total pages scraped: 275


In [43]:
product_detail=[]
for page in product_pages:
    for product in page:
        id=product.get("id")
        if not id:
            continue

        Ean_code= product.get("ean_code")
        if not Ean_code:
            continue

        price=product["pricing"]["discount"].get("mrp")
        if not price:
            continue
            
        Image_link=product.get("images")
        if not Image_link:
            continue
            
        Brand=product["brand"].get("name")
        if not Brand:
            continue
            
        title=product.get("desc")
        if not title:
            continue
            
        w=product.get("w") if product.get("w") else "Unavailable"
        if "x" in w or w=="Unavailable":
            continue
            
        availability=product["availability"].get("avail_status")
        if not availability:
            availability="Unavailable"
            
        tlc,mlc,llc=product["category"].get("tlc_name"), product["category"].get("mlc_name"), product["category"].get("llc_name")
        category=[tlc,mlc,llc]
        
        mag=product.get("magnitude")
        if not mag:
            continue
            
        unit=product.get("unit")
        if not unit:
            continue
            
        w_mag="".join((re.findall("[0-9]",w)))
        w_unit="".join((re.findall("[A-Za-z]",w)))
            
        title=f"({title}){Brand}[{w_mag}]{{{w_unit}}}<{price}>"
                  
        product_detail.append({"Id":id,"EAN code":Ean_code,"Title":title,"Brand":Brand,"Magnitude":mag,"Unit":unit,"w_mag":w_mag,"w_unit":w_unit,"Price":price,"Availability":availability,"Category":category,"Image":Image_link})
        
print("done")
# print(product_detail)

done


In [44]:
keys=product_detail[0].keys()

with open("scraped_data_final.csv","w", newline='', encoding='utf-8') as output_file:
    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
    dict_writer.writeheader()  # Uncomment if writing the header for the first time
    dict_writer.writerows(product_detail)

In [45]:
import pandas as pd
df=pd.read_csv("scraped_data_final.csv")
df

,Id,EAN code,Title,Brand,Magnitude,Unit,w_mag,w_unit,Price,Availability,Category,Image
0,40075197,890537007116,(Basmati Rice/Basmati Akki - Rozana Super 90)D...,Daawat,5000.0,g,5.0,kg,495.0,1,"['Foodgrains, Oil & Masala', 'Rice & Rice Prod...",[{'s': 'https://www.bigbasket.com/media/upload...
1,147492,890188716598,(Gold Refined Cooking oil | Blended Rice Bran ...,Saffola,2000.0,ml,2.0,L,375.0,1,"['Foodgrains, Oil & Masala', 'Edible Oils & Gh...",[{'s': 'https://www.bigbasket.com/media/upload...
2,20005214,8904043926797,(Fine Besan/Kadale Hittu - 100% Chana Dal)Tata...,Tata Sampann,1000.0,g,1.0,kg,173.0,1,"['Foodgrains, Oil & Masala', 'Atta, Flours & S...",[{'s': 'https://www.bigbasket.com/media/upload...
3,30010377,2000539,(Urad Dal/Uddina Bele - Split)bb Popular[1]{kg...,bb Popular,1000.0,g,1.0,kg,220.0,1,"['Foodgrains, Oil & Masala', 'Dals & Pulses', ...",[{'s': 'https://www.bigbasket.com/media/upload...
4,10000537,10000537,(Walnut/Akhrot - Kernels)bb Popular[200]{g}<360>,bb Popular,200.0,g,200.0,g,360.0,1,"['Foodgrains, Oil & Masala', 'Dry Fruits', 'Ot...",[{'s': 'https://www.bigbasket.com/media/upload...
...,...,...,...,...,...,...,...,...,...,...,...,...
12375,40106288,40106288,(Wheat Lokwan - Premium)bb Royal[10]{kg}<800>,bb Royal,10000.0,g,10.0,kg,800.0,1,"['Foodgrains, Oil & Masala', 'Rice & Rice Prod...",[{'s': 'https://www.bigbasket.com/media/upload...
12376,40168533,40168533,(Organic - Kuttu Atta - Buckwheat)bb Royal[500...,bb Royal,500.0,g,500.0,g,99.0,1,"['Foodgrains, Oil & Masala', 'Organic Staples'...",[{'s': 'https://www.bigbasket.com/media/upload...
12377,40041445,40041445,(Premium Diwali Gift Box)bb Royal[250]{g}<499>,bb Royal,250.0,g,250.0,g,499.0,1,"['Foodgrains, Oil & Masala', 'Organic Staples'...",[{'s': 'https://www.bigbasket.com/media/upload...
12378,40301934,769807238381,(Bombay Pav Bhaji Cooking Paste)CURRYiT[120]{g...,CURRYiT,120.0,g,120.0,g,140.0,1,"['Snacks & Branded Foods', 'Ready To Cook & Ea...",[{'s': 'https://www.bigbasket.com/media/upload...
